In [9]:
%run ../utils/init_env.py

In [10]:
import sqlite3
import json
import os
import pandas as pd
import nltk
nltk.download('punkt_tab')

import evaluation as spider

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/atissera/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
DB_PATH = "/Users/atissera/Developer/repos/tesis/spider_data/test_database"
PREDICTIONS = "/Users/atissera/Developer/repos/tesis/spider_data/test.json"

In [ ]:

kmaps = spider.build_foreign_key_map_from_json("../spider_data/test_tables.json")
spider.evaluate(gold="../data/gold_queries.sql", predict="../data/predicted_queries.sql", db_dir="../spider_data/test_database", etype="all", kmaps=kmaps)

eval_err_num:1
medium pred: SELECT s.bio_data, sa.address_details FROM Students s JOIN Students_Addresses sa ON s.student_id = sa.student_id This query retrieves the bio data from the `Students` table and joins it with the `Students_Addresses` table to get the address details for each student.
medium gold: SELECT T1.address_details ,  T3.bio_data FROM Addresses AS T1 JOIN Students_Addresses AS T2 ON T1.address_id  =  T2.address_id JOIN Students AS T3 ON T2.student_id  =  T3.student_id

hard pred: SELECT AVG(SalePrice) FROM Book WHERE Author IN ( SELECT idAuthor FROM Author WHERE Name = 'George Orwell' )
hard gold: SELECT avg(saleprice) FROM Book AS T1 JOIN Author_book AS T2 ON T1.isbn  =  T2.isbn JOIN Author AS T3 ON T2.Author  =  T3.idAuthor WHERE T3.name  =  "George Orwell"

eval_err_num:2
hard pred: SELECT DISTINCT b1.Code FROM Boxes b1, Boxes b2 WHERE b1.Value > ALL ( SELECT b3.Value FROM Boxes b3 WHERE b3.Contents = 'Rocks' ) AND b1.Code != ALL ( SELECT b4.Code FROM Boxes b4 WHERE

In [1]:

# with open(PREDICTIONS) as f:
#     predictions = json.load(f)

# for item in predictions[:5]:
#     db_id = item["db_id"]
#     query = item["query"]
#     db_file = os.path.join(DB_PATH, db_id, db_id + ".sqlite")
#     if not os.path.exists(db_file):
#         print(f"❌ Database file not found: {db_file}")
#         continue

#     try:
#         conn = sqlite3.connect(db_file)
#         cursor = conn.cursor()
#         cursor.execute(query)
#         columns = [description[0] for description in cursor.description]  # get column names
#         rows = cursor.fetchall()
#         conn.close()

#         result_df = pd.DataFrame(rows, columns=columns)
#         print(f"📌 Database: {db_id} | Query:\n{query}")
#         display(result_df)
#     except Exception as e:
#         print(f"❌ Error running query on {db_id}: {query}")
#         print(e)